In [6]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import OpenAIEmbeddings


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [2]:
loader = DirectoryLoader(
    "./articles",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
documents = loader.load()

In [3]:
len(documents)

5

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
texts = text_splitter.split_documents(documents)


In [9]:
!pip install InstructorEmbedding


In [17]:
persist_directory = 'db'
from langchain.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl",      # or instructor-base, -large, -small…
    model_kwargs={
        "device": "cpu",                   # or "cpu"
    }
)

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embeddings,
                                 persist_directory=persist_directory)


No sentence-transformers model found with name hkunlp/instructor-xl. Creating a new one with mean pooling.
`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


In [18]:
# persiste the db to disk
vectordb.persist()
vectordb = None

C:\Users\adnan\AppData\Local\Temp\ipykernel_30676\2083128404.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [19]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embeddings)

C:\Users\adnan\AppData\Local\Temp\ipykernel_30676\1600266097.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


In [20]:
retriever = vectordb.as_retriever()

In [21]:
docs = retriever.get_relevant_documents("For Whom cybersecurity has emerged as a critical priority?")

C:\Users\adnan\AppData\Local\Temp\ipykernel_30676\3369911407.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("For Whom cybersecurity has emerged as a critical priority?")
`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


In [22]:
len(docs)

4

In [23]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [26]:
retriever.search_type

'similarity'

In [27]:
retriever.search_kwargs

{'k': 2}

In [30]:
# create the chain to answer questions 
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

llm = Ollama(model="llama3.2")

# 2. Build the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                      # pass the LLM instance, not "llama3.2"
    chain_type="stuff",           # or "map_reduce", "refine", etc.
    retriever=retriever,          # e.g. vectordb.as_retriever()
    return_source_documents=True  # include the source docs in the output
)

C:\Users\adnan\AppData\Local\Temp\ipykernel_30676\2578228853.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [33]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [34]:
query = "For Whom cybersecurity has emerged as a critical priority?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


For individuals, businesses, and governments.


Sources:
articles\cybersecurity_in_digital.txt
articles\cybersecurity_in_digital.txt


In [35]:
query = "What are the human activities that cause shift in termprature?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


Burning of fossil fuels (such as coal, oil, and gas) and deforestation are two examples mentioned in the context. However, I don't know all the specific human activities that contribute to climate change.


Sources:
articles\Impact_of_climateChange.txt
articles\Impact_of_climateChange.txt
